In [22]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import io
# check the versions of libraries
print("Tensorflow Version: {}".format(tf.__version__))
print("NumPy Version: {}".format(np.__version__))
print("Tensorflow Datasets Version: {}".format(tfds.__version__))

Tensorflow Version: 2.1.0
NumPy Version: 1.18.2
Tensorflow Datasets Version: 3.2.1


In [2]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [3]:
train_data, test_data = imdb['train'], imdb['test']

In [4]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

In [5]:
for sentences, labels in train_data:
    training_sentences.append(str(sentences.numpy()))
    training_labels.append(labels.numpy())

In [6]:
for sentences, labels in test_data:
    testing_sentences.append(str(sentences.numpy()))
    testing_labels.append(labels.numpy())

In [7]:
training_labels_final = np.array(training_labels)
test_labels_final = np.array(testing_labels)

In [8]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = "post"
oov_tok = "<OOV>"

In [9]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

In [10]:
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [11]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [15]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [16]:
num_epochs = 10
model.fit(padded,
         training_labels_final,
         epochs=num_epochs,
         validation_data=(testing_padded, test_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 8s 303us/sample - loss: 0.5679 - accuracy: 0.7160 - val_loss: 0.3933 - val_accuracy: 0.8393
Epoch 2/10
25000/25000 [==============================] - 6s 260us/sample - loss: 0.3405 - accuracy: 0.8587 - val_loss: 0.3381 - val_accuracy: 0.8541
Epoch 3/10
25000/25000 [==============================] - 6s 232us/sample - loss: 0.2774 - accuracy: 0.8870 - val_loss: 0.3298 - val_accuracy: 0.8574
Epoch 4/10
25000/25000 [==============================] - 5s 216us/sample - loss: 0.2416 - accuracy: 0.9045 - val_loss: 0.3387 - val_accuracy: 0.8544
Epoch 5/10
25000/25000 [==============================] - 7s 283us/sample - loss: 0.2150 - accuracy: 0.9183 - val_loss: 0.3638 - val_accuracy: 0.8478
Epoch 6/10
25000/25000 [==============================] - 5s 212us/sample - loss: 0.1943 - accuracy: 0.9282 - val_loss: 0.3818 - val_accuracy: 0.8450
Epoch 7/10
25000/25000 [==========================

In [19]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [21]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

In [23]:
out_vector = io.open('vectors.tsv', 'w', encoding='utf-8')
out_meta = io.open('metadata.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_meta.write(word + '\n')
    out_vector.write('\t'.join([str(x) for x in embeddings]) + '\n')
    
out_vector.close()
out_meta.close()


In [24]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 68, 105, 12, 7, 491, 1216]]
